In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter
from scipy import misc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Image
imagePath = 'details.jpg'
image = Image.open(imagePath)
grayscaleImage = image.convert('F')
plt.imshow(grayscaleImage, cmap='gray')
# plt.axis('off')
plt.show()

In [ ]:
# Variables
sigma = 1.6
s = 2
k = 2 ** (1/s)
noOfOctaves = 4
imagesPerOctave = s + 3

In [ ]:
# Initialize arrays of values which will be used as a blur parameter to the gaussian filter
blurValues = np.zeros((noOfOctaves, imagesPerOctave))
val = sigma/2
for i in range(noOfOctaves):
    for j in range(imagesPerOctave):
        blurValues[i][j] = val
        val = val*k
    val = blurValues[i][2]
blurValues

In [ ]:
# Set up images of different scales for each octave
ImageSize2 = np.array(grayscaleImage) # original-size image
ImageSize1 = misc.imresize(ImageSize2, 200, 'bilinear')
ImageSize4 = misc.imresize(ImageSize1, 50, 'bilinear')
ImageSize8 = misc.imresize(ImageSize2, 50, 'bilinear')

fig = plt.figure(figsize=(20, 20))
fig.add_subplot(1, 4, 1)
plt.imshow(ImageSize1, cmap='gray')
fig.add_subplot(1, 4, 2)
plt.imshow(ImageSize2, cmap='gray')
fig.add_subplot(1, 4, 3)
plt.imshow(ImageSize4, cmap='gray')
fig.add_subplot(1, 4, 4)
plt.imshow(ImageSize8, cmap='gray')
plt.show()


In [ ]:
# Calculate Gaussian blurred images for each octave
scaledImages = [ImageSize1, ImageSize2, ImageSize4, ImageSize8]

octaves = []
octave1 = np.zeros((ImageSize1.shape[0], ImageSize1.shape[1], imagesPerOctave))
octave2 = np.zeros((ImageSize2.shape[0], ImageSize2.shape[1], imagesPerOctave))
octave3 = np.zeros((ImageSize4.shape[0], ImageSize4.shape[1], imagesPerOctave))
octave4 = np.zeros((ImageSize8.shape[0], ImageSize8.shape[1], imagesPerOctave))
octaves.append(octave1)
octaves.append(octave2)
octaves.append(octave3)
octaves.append(octave4)

for i in range(noOfOctaves):
    for j in range(imagesPerOctave):
        octaves[i][:,:,j]= gaussian_filter(scaledImages[i], sigma=blurValues[i][j])

In [ ]:
# Calculate differences of Gaussians(DoG) for each octave
DoGs = []
DoGImageNo = imagesPerOctave-1
DoG1 = np.zeros((ImageSize1.shape[0], ImageSize1.shape[1], DoGImageNo))
DoG2 = np.zeros((ImageSize2.shape[0], ImageSize2.shape[1], DoGImageNo))
DoG3 = np.zeros((ImageSize4.shape[0], ImageSize4.shape[1], DoGImageNo))
DoG4 = np.zeros((ImageSize8.shape[0], ImageSize8.shape[1], DoGImageNo))
DoGs.append(DoG1)
DoGs.append(DoG2)
DoGs.append(DoG3)
DoGs.append(DoG4)

for i in range(noOfOctaves):
    for j in range(DoGImageNo):
        DoGs[i][:,:,j] = octaves[i][:,:,j+1] - octaves[i][:,:,j]

In [ ]:
plt.imshow(DoGs[0][:,:,0], cmap='gray')
plt.show()

In [79]:
# Locate extrema in DoG images
# Filter the extrema, removing keypoints along edges
contrastThreshold = 0.03
keypoints = []
eigenValMagRatio = 10

for octave in range(noOfOctaves):
    for i in range(1, DoGImageNo - 1):
        for y in range(1, DoGs[octave][:,:,i].shape[0] - 1):
            for x in range(1, DoGs[octave][:,:,i].shape[1] - 1):
                currPixVal = DoGs[octave][y, x, i]
                if (np.absolute(currPixVal) / 255)  < contrastThreshold:
                    continue
                isMin = True
                isMax = True
                for r in range(-1, 2):
                    for q in range(-1, 2):
                        for p in range(-1, 2):
                            if p==0 and q==0 and r==0:
                                continue
                            neighbour = DoGs[octave][y+q, x+p, i+r]
                            if neighbour <= currPixVal:
                                isMin = False
                            if neighbour >= currPixVal:
                                isMax = False
                            if not isMax and not isMin:
                                break
                        if not isMax and not isMin:
                            break
                    if not isMax and not isMin:
                        break
                if isMax or isMin:
                    Dxx = DoGs[octave][y, x+1, i] - 2*DoGs[octave][y, x, i] + DoGs[octave][y, x-1, i]
                    Dyy = DoGs[octave][y+1, x, i] - 2*DoGs[octave][y, x, i] + DoGs[octave][y-1, x, i]
                    Dxy = (DoGs[octave][y+1, x+1, i] - DoGs[octave][y-1, x+1, i] - DoGs[octave][y+1, x-1, i] + DoGs[octave][y-1, x-1, i]) / 4
                    trace = Dxx + Dyy
                    det = Dxx*Dyy - Dxy**2
                    isKeyPoint = (trace**2 / det) < ((r + 1)**2 / r)

                    if isKeyPoint:
                        keypoint = ((x,y), octave, blurValues[octave][i], 'min' if isMin else 'max', currPixVal)
                        keypoints.append(keypoint)

keypoints, len(keypoints)

([((313, 26), 1, 2.2627416997969525, 'min', -9.9658203125),
  ((322, 28), 1, 2.2627416997969525, 'min', -10.269790649414062),
  ((312, 31), 1, 2.2627416997969525, 'max', 9.582000732421875),
  ((105, 37), 1, 2.2627416997969525, 'min', -8.863555908203125),
  ((127, 343), 1, 2.2627416997969525, 'max', 14.837162017822266),
  ((107, 346), 1, 2.2627416997969525, 'max', 13.624748229980469),
  ((128, 357), 1, 3.2000000000000006, 'max', 11.659523010253906)],
 7)

In [ ]:
# keypoint magnitude and orientation assignment
# Find the DoG which has the nearest blur value compared to that of the current keypoint

In [ ]:
# Keypoint descriptor calculation